In [1]:
import csv
import pandas as pd
import numpy as np

In [2]:
escitalopram_id = "CID100002771" #"CID100002771"
atc1 = "N06AB04"
atc2 = "N06AB10"


In [3]:
dataset_path = "/home/samuel/NYU/BDS/project/meddra/"

def read_tsv_file(file_path, headers = True):
    try:
        data = []
        with open(file_path, 'r', newline='', encoding='utf-8') as file:
            # Using the CSV reader with the tab delimiter
            reader = csv.reader(file, delimiter='\t')

            if headers == True:
                # Reading headers
                headers = next(reader)
                print("Headers:", headers)

            # Reading data rows
            for row in reader:
                data.append(row)

        # Create a Pandas DataFrame using the headers and data
        if headers == True:
            df = pd.DataFrame(data, columns=headers)
        else:
            df = pd.DataFrame(data)
        return df

    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'")
    except Exception as e:
        print(f"An error occurred: {e}")
meddra_file_path = dataset_path + 'meddra_all_se.tsv'
meddra_df = read_tsv_file(meddra_file_path)
escitalopram_sides_table = meddra_df[meddra_df[0] == escitalopram_id]
escitalopram_sides = escitalopram_sides_table[5].unique()
processed_dataset = pd.read_csv("../review_dataset/drugsComTrain_processed.csv")
escitalopram_dataset = processed_dataset[processed_dataset["drugName"] == "Escitalopram"]

Headers: ['CID100000085', 'CID000010917', 'C0000729', 'LLT', 'C0000729', 'Abdominal cramps']


In [4]:
escitalopram_dataset

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,processed_review
203,140891,Escitalopram,Generalized Anxiety Disorde,"""The time I been on lexapro I had mild side ef...",7.0,"April 27, 2016",13,"""the time lexapro mild side effect like yawn i..."
237,214628,Escitalopram,Generalized Anxiety Disorde,"""This drug is pretty great for depression, how...",5.0,"August 14, 2017",7,"""thi drug pretti great depression, howev serio..."
252,141047,Escitalopram,Anxiety,"""Day 2. Taking for major anxiety/ slight depre...",10.0,"October 6, 2015",11,"""day 2. take major anxiety/ slight depression...."
314,142104,Escitalopram,Depression,"""I&#039;ve been on lexapro for several years. ...",8.0,"March 9, 2009",376,"""i'v lexapro sever years. say initi side effec..."
405,141645,Escitalopram,Depression,"""I am a 35 year-old female. I&#039;ve always b...",10.0,"November 3, 2012",81,"""i 35 year-old female. i'v alway melanchol per..."
...,...,...,...,...,...,...,...,...
160596,141107,Escitalopram,Depression,"""It works really good. I also have anxiety and...",9.0,"August 10, 2015",65,"""it work realli good. also anxieti part helps...."
160625,141426,Escitalopram,Anxiety,"""I feel compelled to share my experience becau...",9.0,"January 27, 2014",41,"""i feel compel share experi know other like me..."
160687,140846,Escitalopram,Depression,"""I have taken Lexapro for 10 years. I had seve...",9.0,"April 20, 2016",32,"""i taken lexapro 10 years. sever depression/an..."
160716,141419,Escitalopram,Depression,"""Sexually frustrated me and I was a little bit...",6.0,"February 9, 2014",36,"""sexual frustrat littl bit aggress come onto i..."


In [5]:
import pandas as pd

# Create a DataFrame with side effects and initialize the count to 0
escitalopram_sides_counted = pd.DataFrame({"side_effect_name": escitalopram_sides})

# Iterate through each review
for index, row in escitalopram_dataset.iterrows():
    # Convert the review to lowercase for case-insensitive matching
    review_orig = row["review"].lower()
    review_lower = row["review"].lower()
    
    # Iterate through each side effect
    for side_effect in escitalopram_sides_counted["side_effect_name"]:
        # Replace side effect mentions with an empty string
        review_lower = review_lower.replace(side_effect.lower(), "")
        
    
    # Store the cleaned review in the dataframe
    escitalopram_dataset.loc[index, "cleaned_review"] = review_lower

# Print the first few rows of the dataframe to verify the results

/tmp/ipykernel_58896/2995517786.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset.loc[index, "cleaned_review"] = review_lower


In [6]:
escitalopram_sides_counted = pd.DataFrame(escitalopram_sides, columns=["side_effect_name"]).copy()
escitalopram_sides_counted["count"] = 0
for review in escitalopram_dataset["review"]:
    for side_effect_name in escitalopram_sides_counted["side_effect_name"]:
        if side_effect_name.lower() in review.lower():
            escitalopram_sides_counted.loc[escitalopram_sides_counted["side_effect_name"] == side_effect_name, "count"] += 1
escitalopram_sides_counted.sort_values(by = "count", ascending=False)

,side_effect_name,count
17,Anxiety,785
355,Depression,439
607,Panic attack,213
183,Nausea,122
120,Headache,112
...,...,...
313,Phosphatase alkaline increased,0
314,Blood alkaline phosphatase increased,0
315,Prothrombin level increased,0
316,Electrocardiogram QT prolonged,0


In [7]:
relevant_side_effect = list(escitalopram_sides_counted[escitalopram_sides_counted["count"] > 5]["side_effect_name"])
for relevant_effect in relevant_side_effect:
    escitalopram_dataset.loc[:, relevant_effect] = False
    for index,  review in escitalopram_dataset["review"].items():
        if relevant_effect.lower() in review.lower():
            escitalopram_dataset.loc[index, relevant_effect] = True
reg_df = escitalopram_dataset.drop(columns=["Unnamed: 0", "drugName", "condition", "review", "date", "usefulCount", "processed_review"])


/tmp/ipykernel_58896/436393038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset.loc[:, relevant_effect] = False
/tmp/ipykernel_58896/436393038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset.loc[:, relevant_effect] = False
/tmp/ipykernel_58896/436393038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

/tmp/ipykernel_58896/436393038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset.loc[:, relevant_effect] = False
/tmp/ipykernel_58896/436393038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset.loc[:, relevant_effect] = False
/tmp/ipykernel_58896/436393038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [8]:
import re
for index, row in escitalopram_dataset.iterrows():
    # Convert the review to lowercase for case-insensitive matching
    review_orig = row["review"].lower()
    if "years old" in review_orig:
        age_index = review_orig.find("years old")
        match = re.search(r"\b\d+\b", review_orig[age_index-3:age_index+10])
        if match:
            escitalopram_dataset.loc[index, "age"] = match.group()
            
    if "year-old" in review_orig:
        age_index = review_orig.find("year-old")
        match = re.search(r"\b\d+\b", review_orig[age_index-3:age_index+10])
        if match:
            escitalopram_dataset.loc[index, "age"] = match.group()

    if "years-old" in review_orig:
        age_index = review_orig.find("years-old")
        match = re.search(r"\b\d+\b", review_orig[age_index-3:age_index+10])
        if match:
            escitalopram_dataset.loc[index, "age"] = match.group()
    
    if "year old" in review_orig:
        age_index = review_orig.find("year old")
        match = re.search(r"\b\d+\b", review_orig[age_index-3:age_index+10])
        if match:
            escitalopram_dataset.loc[index, "age"] = match.group()
    
    if "y/o" in review_orig:
        age_index = review_orig.find("y/o")
        match = re.search(r"\b\d+\b", review_orig[age_index-3:age_index+10])
        if match:
            escitalopram_dataset.loc[index, "age"] = match.group()
            
    if "years of age" in review_orig:
        age_index = review_orig.find("years of age")
        match = re.search(r"\b\d+\b", review_orig[age_index-3:age_index+10])
        if match:
            escitalopram_dataset.loc[index, "age"] = match.group()
    
found = escitalopram_dataset[escitalopram_dataset["age"] != "na"].shape[0]
total = escitalopram_dataset.shape[0]
print(f"ratio of found ages: {found/total}")

/tmp/ipykernel_58896/1919217565.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset.loc[index, "age"] = match.group()


ratio of found ages: 0.05959752321981424


In [9]:
import re
escitalopram_dataset["married"] = False
for index, row in escitalopram_dataset.iterrows():
    # Convert the review to lowercase for case-insensitive matching
    review_orig = row["review"].lower()
    if "husband" in review_orig:
        escitalopram_dataset.loc[index, "married"] = True
    
    if "wife" in review_orig:
        escitalopram_dataset.loc[index, "married"] = True
    
    if "married" in review_orig:
        escitalopram_dataset.loc[index, "married"] = True
    
    if "marriage" in review_orig:
        escitalopram_dataset.loc[index, "married"] = True
    
    if "spouse" in review_orig:
        escitalopram_dataset.loc[index, "married"] = True
    
found = escitalopram_dataset[escitalopram_dataset["married"] != False].shape[0]
total = escitalopram_dataset.shape[0]
print(f"ratio of found married: {found/total}")

ratio of found married: 0.053405572755417956


/tmp/ipykernel_58896/275061147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset["married"] = False


In [10]:
import re
escitalopram_dataset.loc[:,"children"] = False
for index, row in escitalopram_dataset.iterrows():
    # Convert the review to lowercase for case-insensitive matching
    review_orig = row["review"].lower()
    if "child" in review_orig:
        escitalopram_dataset.loc[index, "children"] = True
    
    if "children" in review_orig:
        escitalopram_dataset.loc[index, "children"] = True
    
    if "kid" in review_orig:
        escitalopram_dataset.loc[index, "children"] = True
    
    if "kids" in review_orig:
        escitalopram_dataset.loc[index, "children"] = True
    
found = escitalopram_dataset[escitalopram_dataset["children"] != False].shape[0]
total = escitalopram_dataset.shape[0]
print(f"ratio of found children: {found/total}")

ratio of found children: 0.04411764705882353


/tmp/ipykernel_58896/4181311698.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escitalopram_dataset.loc[:,"children"] = False


In [11]:
relevant_side_effect = list(escitalopram_sides_counted[escitalopram_sides_counted["count"] > 5]["side_effect_name"])

In [12]:
escitalopram_dataset.drop(columns=relevant_side_effect)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,processed_review,cleaned_review,age,married,children
203,140891,Escitalopram,Generalized Anxiety Disorde,"""The time I been on lexapro I had mild side ef...",7.0,"April 27, 2016",13,"""the time lexapro mild side effect like yawn i...","""the time i been on lexapro i had mild side ef...",na,False,False
237,214628,Escitalopram,Generalized Anxiety Disorde,"""This drug is pretty great for depression, how...",5.0,"August 14, 2017",7,"""thi drug pretti great depression, howev serio...","""this drug is pretty great for , however i am ...",na,False,False
252,141047,Escitalopram,Anxiety,"""Day 2. Taking for major anxiety/ slight depre...",10.0,"October 6, 2015",11,"""day 2. take major anxiety/ slight depression....","""day 2. taking for major / slight . 10mg daily...",na,False,False
314,142104,Escitalopram,Depression,"""I&#039;ve been on lexapro for several years. ...",8.0,"March 9, 2009",376,"""i'v lexapro sever years. say initi side effec...","""i&#039;ve been on lexapro for several years. ...",na,False,False
405,141645,Escitalopram,Depression,"""I am a 35 year-old female. I&#039;ve always b...",10.0,"November 3, 2012",81,"""i 35 year-old female. i'v alway melanchol per...","""i am a 35 year-old female. i&#039;ve always b...",35,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
160596,141107,Escitalopram,Depression,"""It works really good. I also have anxiety and...",9.0,"August 10, 2015",65,"""it work realli good. also anxieti part helps....","""it works really good. i also have and for th...",na,False,False
160625,141426,Escitalopram,Anxiety,"""I feel compelled to share my experience becau...",9.0,"January 27, 2014",41,"""i feel compel share experi know other like me...","""i feel compelled to share my experience becau...",na,False,False
160687,140846,Escitalopram,Depression,"""I have taken Lexapro for 10 years. I had seve...",9.0,"April 20, 2016",32,"""i taken lexapro 10 years. sever depression/an...","""i have taken lexapro for 10 years. i had seve...",na,False,False
160716,141419,Escitalopram,Depression,"""Sexually frustrated me and I was a little bit...",6.0,"February 9, 2014",36,"""sexual frustrat littl bit aggress come onto i...","""sexually frustrated me and i was a little bit...",na,False,False


In [92]:
patient_matrix = escitalopram_dataset[escitalopram_dataset["age"] != "na"].drop(columns=["Unnamed: 0", "drugName", "condition", "review", "date", "usefulCount", "processed_review", "rating", "cleaned_review"]+relevant_side_effect)

patient_matrix = patient_matrix.dropna(axis=1, how='any')

patient_matrix

,age,married,children
405,35,False,True
3481,50,False,False
4713,26,False,True
9377,26,False,False
10497,26,False,False
...,...,...,...
147717,23,False,False
151016,26,False,False
151233,37,False,False
152818,34,True,False


In [148]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

patient_matrix = escitalopram_dataset[escitalopram_dataset["age"] != "na"].drop(columns=["Unnamed: 0", "drugName", "condition", "review", "date", "usefulCount", "processed_review", "rating", "cleaned_review"]+relevant_side_effect)

patient_matrix = patient_matrix.dropna(axis=1, how='any')

# patient_matrix = escitalopram_dataset[escitalopram_dataset["age"] != "na"].drop(columns=["Unnamed: 0", "drugName", "condition", "review", "date", "usefulCount", "processed_review"])
# patient_matrix = patient_matrix.dropna(axis=1, how='any')
label_encoder = LabelEncoder()
for col in patient_matrix.columns:
    if patient_matrix[col].dtype == 'object':
        print(col)
        patient_matrix[col] = pd.to_numeric(patient_matrix[col])

# Normalize the data
patient_matrix_norm = (patient_matrix - patient_matrix.mean()) / patient_matrix.std()
patient_matrix_norm["age"] = patient_matrix_norm["age"]*500
patient_matrix_norm["children"] = patient_matrix_norm["children"]/100
#patient_matrix_norm["married"] = patient_matrix_norm["married"]/100



# # Define the query patient
# query_patient = patient_matrix_norm.dropna(axis=1, how='any').iloc[0]

# #alculate cosine similarity between query patient and all other patients
# cosine_similarities = cosine_similarity(patient_matrix_norm.dropna(axis=1, how='any'), [query_patient])

# Define the query patient
query_patient = patient_matrix_norm.iloc[0]

#alculate cosine similarity between query patient and all other patients
cosine_similarities = cosine_similarity(patient_matrix_norm, [query_patient])

patient_matrix["similarity"] = 0.0
patient_matrix_norm["similarity"] = 0.0


for idx, similarity in enumerate(cosine_similarities):
    patient_matrix.iloc[idx,patient_matrix.columns.get_loc("similarity")] = similarity
    patient_matrix_norm.iloc[idx,patient_matrix.columns.get_loc("similarity")] = similarity
    
patient_matrix.sort_values(by = "similarity", ascending = False)

age


,age,married,children,similarity
405,35,False,True,1.000000
87012,36,False,False,1.000000
73430,36,False,False,1.000000
53921,36,False,False,1.000000
151233,37,False,False,1.000000
...,...,...,...,...
20291,15,False,False,-0.999996
122126,14,False,False,-0.999996
114762,10,False,True,-0.999996
45960,9,False,False,-0.999996


In [165]:
recommends = escitalopram_dataset.loc[patient_matrix.sort_values(by = "similarity", ascending = False).head(20).index]
recommends

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,processed_review,cleaned_review,Anxiety,...,Irritability,Lethargy,Night sweats,Panic attack,Upset stomach,Crying,Anxiety attack,age,married,children
405,141645,Escitalopram,Depression,"""I am a 35 year-old female. I&#039;ve always b...",10.0,"November 3, 2012",81,"""i 35 year-old female. i'v alway melanchol per...","""i am a 35 year-old female. i&#039;ve always b...",True,...,False,False,False,False,False,False,False,35,False,True
87012,141209,Escitalopram,Anxiety,"""I am a 36 year old female. I have been on Lex...",9.0,"April 24, 2015",53,"""i 36 year old female. lexapro 4 month now, de...","""i am a 36 year old female. i have been on lex...",True,...,False,False,False,False,False,False,False,36,False,False
73430,141674,Escitalopram,Depression,"""I&#039;m a healthy 36 year old male. The firs...",8.0,"January 15, 2013",68,"""i'm healthi 36 year old male. first anti-depr...","""i&#039;m a healthy 36 year old male. the firs...",False,...,False,False,False,False,False,False,False,36,False,False
53921,141918,Escitalopram,Depression,"""I am a 36 year old single female. I have rai...",10.0,"February 3, 2010",44,"""i 36 year old singl female. rais 2 boy 16 yea...","""i am a 36 year old single female. i have rai...",True,...,False,False,False,False,False,False,True,36,False,False
151233,141846,Escitalopram,Generalized Anxiety Disorde,"""Lexapro has been incredible for me. I am a 37...",10.0,"October 20, 2010",170,"""lexapro incred me. 37 year old man. 5 1/2 wee...","""lexapro has been incredible for me. i am a 37...",False,...,False,False,False,False,False,False,False,37,False,False
110831,141556,Escitalopram,Generalized Anxiety Disorde,"""I am 39 year old suffering from anxiety and p...",7.0,"June 4, 2013",7,"""i 39 year old suffer anxieti panic disord 8 y...","""i am 39 year old suffering from and panic di...",True,...,False,False,False,True,False,False,False,39,False,False
48779,141566,Escitalopram,Anxiety,"""Lexapro saved my life pure and simple. I was ...",10.0,"May 20, 2013",135,"""lexapro save life pure simple. 42 year old wo...","""lexapro saved my life pure and simple. i was ...",True,...,False,False,False,True,False,False,False,42,False,False
46487,141700,Escitalopram,Generalized Anxiety Disorde,"""I am 43 years old and after my son left for c...",10.0,"November 9, 2011",58,"""i 43 year old son left colleg 6 yrs. ago life...","""i am 43 years old and after my son left for c...",True,...,False,False,False,False,False,False,False,43,False,False
50689,141668,Escitalopram,Depression,"""I am a 43 year old, otherwise healthy male, w...",10.0,"February 3, 2013",75,"""i 43 year old, otherwis healthi male, reactiv...","""i am a 43 year old, otherwise healthy male, w...",True,...,False,False,False,False,False,False,False,43,False,False
39533,141805,Escitalopram,Anxiety,"""I&#039;m a 47 year old female, been on 10mg L...",8.0,"December 30, 2010",16,"""i'm 47 year old female, 10mg lexapro 5 year a...","""i&#039;m a 47 year old female, been on 10mg l...",True,...,False,False,False,False,False,False,False,47,False,True


In [166]:
recommends[relevant_side_effect].sum().sort_values(ascending=False)

Anxiety             16
Depression           9
Panic attack         3
Insomnia             2
Dry mouth            2
Pain                 1
Upset stomach        1
Anger                1
Sweating             1
Suicide              1
Anxiety attack       1
Rash                 1
Fatigue              1
Headache             1
Nightmare            0
Palpitations         0
Crying               0
Constipation         0
Dizziness            0
Night sweats         0
Lethargy             0
Irritability         0
Hunger               0
Fibromyalgia         0
Myalgia              0
Mania                0
Migraine             0
Mood swings          0
Yawning              0
Major depression     0
Chest pain           0
Vomiting             0
Tremor               0
Shock                0
Nausea               0
Tension              0
Trauma               0
dtype: int64